In [1]:
import numpy as np
import matplotlib.pyplot as plt
from __future__ import division

In [2]:
# form mzp6.ipnyb
def draw_system(method, t_0, y_0s, approx, exact = None, labels=None, step = 0.1):
    count = np.ceil(10.0/step)
    X = np.linspace(t_0,t_0+10,count)
    Y = np.zeros(shape=(len(approx),X.shape[0]))
    if exact:
        RY = np.asarray([y(X) if y is not None else None for y in exact])
    Y[:,0] = np.asarray(y_0s)
    
    for i in np.arange(1,count):
        Y_updates = method(tn=X[i-1],yn=Y[:,i-1],functions=approx,h=step)
        if isinstance(Y_updates,tuple):
            Y[:,i] = Y[:,i-1] + Y_updates[1]
        else:
            Y[:,i] = Y[:,i-1] + Y_updates
    
    figure_count = len(approx)
    if exact:
        figure_count += len(filter(lambda x: x is not None,exact))
    figure = 1
    plt.figure(figsize = (9,figure_count*3))
    for i in range(len(approx)):
        ax = plt.subplot(figure_count,1,figure)
        if labels and labels[i]:
            ax.set_title(labels[i])
        plt.plot(X,Y[i], color = 'blue', label ='Approx')
        if exact and exact[i]:
            plt.plot(X,RY[i], color = 'red', linestyle='--', label='Exact')
        plt.legend(loc = 'upper left')
        figure += 1

        if exact and exact[i]:
            plt.subplot(figure_count,1,figure)
            plt.plot(X,np.abs(RY[i]-Y[i]), color = 'blue', label= 'Global error')
            plt.legend(loc = 'upper left')
            figure+=1
            
    plt.show()

In [3]:
# form mzp6.ipnyb
class SystemSolver():
    def __init__(self,updates = None):
        if isinstance(updates,list):
            max_len = max([len(x) for x in updates])
            [x.extend([0]*(max_len-len(x))) for x in updates]
            self.updates = np.asarray(updates, dtype = np.float64)
        else:
            self.updates = updates
        
    @property
    def update_count(self):
        return min(self.updates.shape)-1
    
    def __call__(self,tn,yn,functions,h):
        k = np.zeros(shape=(self.update_count,len(functions)))
        for i in range(self.update_count):
            a = self.updates[i,0]
            b = self.updates[i,1:]
            upd = np.sum(b[:,np.newaxis]*k,axis = 0)
            for j in range(len(functions)):
                k[i,j]= h * functions[j](tn+a*h, yn+upd)
        c = self.updates[self.update_count,1:]
        if self.updates.shape[0]>self.updates.shape[1]:
            c_= self.updates[self.update_count+1,1:]
            return np.sum(c[:,np.newaxis]*k, axis = 0), np.sum(c_[:,np.newaxis]*k, axis = 0)
        return np.sum(c[:,np.newaxis]*k, axis = 0)

In [4]:
fehlberg_4_5 = SystemSolver([[0],
                            [1/4,1/4],
                            [3/8,3/32,9/32],
                            [12/13,1932/2197,-7200/2197,7296/2197],
                            [1,439/216,-8,3680/513,-845/4104],
                            [1/2,-8/27,2,-3544/2565,1859/4104,-11/40],
                            [0,25/216,0,1408/2565,2197/4104,-1/5,0],
                            [0,16/135,0,6656/12825,28561/56430,-9/50,2/55]])
fehlberg_4_5.ranks = (4,5)

In [5]:
def get_spring(start,end,count,size = 1.):
    start = np.asarray(start)
    end = np.asarray(end)
    length = np.sqrt(np.sum((end-start)**2))
    flat_space = length/6.
    x = np.linspace(flat_space,length-flat_space,count*4+1)
    length_needed = 2*np.pi*count 
    s = size*np.sin((length_needed/(length-flat_space*2))*x)
    rotations = (end-start)/length
    rotation_matrix = np.array([[rotations[0],-rotations[1]],[rotations[1],rotations[0]]])
    first_flat = np.array([[0],[0]])
    second_flat = np.array([[length],[0]])
    return rotation_matrix.dot(np.hstack((first_flat, np.array([x,s]), second_flat))) + start[:,np.newaxis]

Dwie spreżyny

In [6]:
k_1 = 4
k_2 = 10
L = 3
m = 3

In [7]:
x_0 = 0
v_0 = 3

In [ ]:
functions = [
    lambda t,ys: ys[1],
    lambda t,ys: (k_1 - k_2)/m*ys[0]
]

In [ ]:
draw_system(fehlberg_4_5,
            0.,
            y_0s=[x_0,v_0],
            approx = functions,
            labels = ["x(t)","v(t)"])

In [ ]:
Y = np.zeros(shape=(2,))

In [ ]:
def double_spring(method, y_0s, approx, step = 0.1, filename = None):
    import matplotlib.animation as animation
    
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111)
    ax.grid()
    plt.ylim(-20,20)
    plt.xlim(-20,20)

    spring1, = ax.plot([],[])
    spring2, = ax.plot([],[])
    
    time_template = 'time = %.1fs'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    
    mass, = ax.plot([],[], marker='o', markersize = 10.)
    
    def init():
        global Y
        Y = y_0s
        spring1.set_data([],[])
        spring2.set_data([],[])
        time_text.set_text('')
        mass.set_data([],[])
        
        return spring1, spring2, time_text, mass
    
    def animate(t):
        global Y
        y_1, y_2 = method(tn=t*step,yn=Y,functions=approx,h=step)
        Y += y_2
        new_x = Y[0]
        spring = get_spring([-10,0],[new_x,0],6)
        spring1.set_data(spring[0],spring[1])
        spring = get_spring([new_x,0],[10,0],6)
        spring2.set_data(spring[0],spring[1])
        mass.set_data([new_x],[0])

        time_text.set_text(time_template % (t*step))
        return spring1, spring2, time_text, mass
    
    ani = animation.FuncAnimation(fig, animate, np.arange(400),
                              interval=25, blit=True, init_func=init)

    if filename:
        ani.save(filename + '.mp4', fps=30)
    plt.show()

In [ ]:
k_1 = 5
k_2 = 10
L = 3
m = 3

In [ ]:
x_0 = 2
v_0 = 3

In [ ]:
double_spring(method=fehlberg_4_5,
               y_0s=[x_0,v_0],
               approx= functions,
               step=0.05)

In [ ]:
def triple_spring(method, y_0s, approx, step = 0.1, filename = None):
    import matplotlib.animation as animation
    
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111)
    ax.grid()
    plt.ylim(-20,20)
    plt.xlim(-20,20)

    spring1, = ax.plot([],[])
    spring2, = ax.plot([],[])
    spring3, = ax.plot([],[])
    
    time_template = 'time = %.1fs'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    
    mass, = ax.plot([],[], linestyle='None', marker='o', markersize = 10.)
    
    points = (-15,-5,5,15)
    
    def init():
        global Y
        Y = y_0s
        spring1.set_data([],[])
        spring2.set_data([],[])
        spring3.set_data([],[])
        time_text.set_text('')
        mass.set_data([],[])
        
        return spring1, spring2, spring3, time_text, mass
    
    def animate(t):
        global Y
        y_1, y_2 = method(tn=t*step,yn=Y,functions=approx,h=step)
        Y += y_2
        fixed_1 = points[0]
        mass_1 = points[1]+Y[0]
        mass_2 = points[2]+Y[2]
        fixed_2 = points[3]
        
        spring = get_spring([fixed_1,0],[mass_1,0],6)
        spring1.set_data(spring[0],spring[1])
        
        spring = get_spring([mass_1,0],[mass_2,0],6)
        spring2.set_data(spring[0],spring[1])
        
        spring = get_spring([mass_2,0],[fixed_2,0],6)
        spring3.set_data(spring[0],spring[1])
        
        mass.set_data([mass_1,mass_2],[0,0])

        time_text.set_text(time_template % (t*step))
        return spring1, spring2, spring3, time_text, mass
    
    ani = animation.FuncAnimation(fig, animate, np.arange(400),
                              interval=25, blit=True, init_func=init)

    if filename:
        ani.save(filename + '.mp4', fps=30)
    plt.show()

In [ ]:
functions = [
    lambda t,ys: ys[1],
    lambda t,ys: (k_1*ys[0]+k_2*(ys[0]-ys[2]))/-m_1,
    lambda t,ys: ys[3],
    lambda t,ys: (k_3*ys[2]+k_2*(ys[2]-ys[0]))/-m_2
]

In [ ]:
k_1 = 2
k_2 = 2
k_3 = 2
m_1 = .2
m_2 = .2

In [ ]:
x_1_0 = 3
v_1_0 = 0
x_2_0 = 0
v_2_0 = 0

In [ ]:
draw_system(fehlberg_4_5,
            0.,
            y_0s=[x_1_0,v_1_0,x_2_0,v_2_0],
            approx = functions,
            labels = ["x_1(t)","v_1(t)","x_2(t)","v_2(t)"])

In [ ]:
triple_spring(method=fehlberg_4_5,
              y_0s=[x_1_0,v_1_0,x_2_0,v_2_0],
              approx= functions,
              step=0.05)

Z oporami ($\psi(v)$) zależnymi od prędkości

$\psi(v) = c  v$

In [ ]:
c = 1

In [ ]:
functions = [
    lambda t,ys: ys[1],
    lambda t,ys: (k_1*ys[0]+k_2*(ys[0]-ys[2]))/-m_1 - c*ys[1],
    lambda t,ys: ys[3],
    lambda t,ys: (k_3*ys[2]+k_2*(ys[2]-ys[0]))/-m_2 - c*ys[3]
]

In [ ]:
triple_spring(method=fehlberg_4_5,
              y_0s=[x_1_0,v_1_0,x_2_0,v_2_0],
              approx= functions,
              step=0.05)

$\psi(v) = sgn(v) c  v^2$

In [ ]:
c = 0.1

In [ ]:
x_1_0 = 20
v_1_0 = 0
x_2_0 = -10
v_2_0 = 0

In [ ]:
functions = [
    lambda t,ys: ys[1],
    lambda t,ys: (k_1*ys[0]+k_2*(ys[0]-ys[2]))/-m_1 - c*np.sign(ys[1])*ys[1]**2,
    lambda t,ys: ys[3],
    lambda t,ys: (k_3*ys[2]+k_2*(ys[2]-ys[0]))/-m_2 - c*np.sign(ys[3])*ys[3]**2
]

In [ ]:
triple_spring(method=fehlberg_4_5,
              y_0s=[x_1_0,v_1_0,x_2_0,v_2_0],
              approx= functions,
              step=0.05)

Dwie spręzyny - ruch w dwóch wymiarach

In [ ]:
def double_spring_2d(method, y_0s, approx, step = 0.1, filename = None):
    import matplotlib.animation as animation
    
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111)
    ax.grid()
    plt.ylim(-20,20)
    plt.xlim(-20,20)

    
    old_x = []
    old_y = []
    
    history, = ax.plot([],[],linestyle='--')
    
    spring1, = ax.plot([],[])
    spring2, = ax.plot([],[])
    
    time_template = 'time = %.1fs'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    
    mass, = ax.plot([],[], marker='o', markersize = 10.)
    
    
    
    def init():
        global Y
        Y = y_0s
        spring1.set_data([],[])
        spring2.set_data([],[])
        time_text.set_text('')
        mass.set_data([],[])
        history.set_data([],[])
    
        return history, spring1, spring2, time_text, mass
    
    def animate(t):
        global Y
        y_1, y_2 = method(tn=t*step,yn=Y,functions=approx,h=step)
        Y += y_2
        new_x = Y[0]
        new_y = Y[2]
        old_x.append(new_x)
        old_y.append(new_y)
        spring = get_spring([-10,0],[new_x,new_y],6)
        spring1.set_data(spring[0],spring[1])
        spring = get_spring([new_x,new_y],[10,0],6)
        spring2.set_data(spring[0],spring[1])
        mass.set_data([new_x],[new_y])
        history.set_data([old_x],[old_y])

        time_text.set_text(time_template % (t*step))
        return history, spring1, spring2, time_text, mass
    
    ani = animation.FuncAnimation(fig, animate, np.arange(400),
                              interval=25, blit=True, init_func=init)

    if filename:
        ani.save(filename + '.mp4', fps=30)
    plt.show()

In [ ]:
k = 2
L = 10
m = 3
g = 9.81

In [ ]:
a = lambda ys: np.sqrt((L + ys[0])**2 + ys[2]**2)
b = lambda ys: np.sqrt((L - ys[0])**2 + ys[2]**2)
functions = [
    lambda t,ys: ys[1],
    lambda t,ys: ( k * ( (L+ys[0]) *( L - a(ys)) / a(ys) - (L-ys[0])*(L - b(ys)) / b(ys) ) )/m,
    lambda t,ys: ys[3],
    lambda t,ys: -g + 1/m * ( k* ys[2] * ( ( L - a(ys) ) / a(ys) + (L - b(ys)) / b(ys) ))
]

In [ ]:
x_0 = 0
v_x_0 = 0
y_0 = 0
v_y_0 = 0

In [ ]:
draw_system(fehlberg_4_5,
            0.,
            y_0s=[x_0,v_x_0,y_0,v_y_0],
            approx = functions,
            labels = ["x(t)","v_x(t)","y(t)","v_y(t)"])

In [ ]:
double_spring_2d(method=fehlberg_4_5,
              y_0s=[x_0,v_x_0,y_0,v_y_0],
              approx= functions,
              step=0.05)

In [ ]:
x_0 = 0
v_x_0 = 15
y_0 = 0
v_y_0 = 15

In [ ]:
k = 5

In [ ]:
double_spring_2d(method=fehlberg_4_5,
              y_0s=[x_0,v_x_0,y_0,v_y_0],
              approx= functions,
              step=0.05)

Z oporami ruchu

In [ ]:
c = .05

In [ ]:
a = lambda ys: np.sqrt((L + ys[0])**2 + ys[2]**2)
b = lambda ys: np.sqrt((L - ys[0])**2 + ys[2]**2)
functions = [
    lambda t,ys: ys[1],
    lambda t,ys: ( k * ( (L+ys[0]) *( L - a(ys)) / a(ys) - (L-ys[0])*(L - b(ys)) / b(ys) ) )/m - np.sign(ys[1]) * c * ys[1]**2,
    lambda t,ys: ys[3],
    lambda t,ys: -g + 1/m * ( k* ys[2] * ( ( L - a(ys) ) / a(ys) + (L - b(ys)) / b(ys) )) - np.sign(ys[3]) * c * ys[3]**2
]

In [ ]:
double_spring_2d(method=fehlberg_4_5,
              y_0s=[x_0,v_x_0,y_0,v_y_0],
              approx= functions,
              step=0.05)

System z czterema spręzynami

In [ ]:
def spring_node(method, y_0s, approx, step = 0.1, filename = None):
    import matplotlib.animation as animation
    
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111)
    ax.grid()
    plt.ylim(-20,20)
    plt.xlim(-20,20)

    
    old_x = []
    old_y = []
    
    history, = ax.plot([],[],linestyle='--')
    
    spring1, = ax.plot([],[])
    spring2, = ax.plot([],[])
    spring3, = ax.plot([],[])
    spring4, = ax.plot([],[])
    
    time_template = 'time = %.1fs'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    
    mass, = ax.plot([],[], marker='o', markersize = 10.)
    
    
    def init():
        global Y
        Y = y_0s
        spring1.set_data([],[])
        spring2.set_data([],[])
        time_text.set_text('')
        mass.set_data([],[])
        history.set_data([],[])
        spring3.set_data([],[])
        spring4.set_data([],[])
    
        return history, spring1, spring2, spring3, spring4, time_text, mass
    
    def animate(t):
        global Y
        y_1, y_2 = method(tn=t*step,yn=Y,functions=approx,h=step)
        Y += y_2
        new_x = Y[0]
        new_y = Y[2]
        old_x.append(new_x)
        old_y.append(new_y)
        spring = get_spring([-10,0],[new_x,new_y],6)
        spring1.set_data(spring[0],spring[1])
        spring = get_spring([new_x,new_y],[10,0],6)
        spring2.set_data(spring[0],spring[1])
        spring = get_spring([0,10],[new_x,new_y],6)
        spring3.set_data(spring[0],spring[1])
        spring = get_spring([new_x,new_y],[0,-10],6)
        spring4.set_data(spring[0],spring[1])
        mass.set_data([new_x],[new_y])
        history.set_data([old_x],[old_y])

        time_text.set_text(time_template % (t*step))
        return history, spring1, spring2, spring3,spring4, time_text, mass
    
    ani = animation.FuncAnimation(fig, animate, np.arange(400),
                              interval=25, blit=True, init_func=init)

    if filename:
        ani.save(filename + '.mp4', fps=30)
    plt.show()

In [ ]:
c_ = 0.2

In [ ]:
a = lambda ys: np.sqrt((L + ys[0])**2 + ys[2]**2)
b = lambda ys: np.sqrt((L - ys[0])**2 + ys[2]**2)
c = lambda ys: np.sqrt((ys[0]) + (L + ys[2])**2)
d = lambda ys: np.sqrt((ys[0]) + (L - ys[2])**2)
functions = [
    lambda t,ys: ys[1],
    lambda t,ys: ( k * ( (L+ys[0]) *( L - a(ys)) / a(ys) - (L-ys[0])*(L - b(ys)) / b(ys) + \
                   ys[0] *( (L-c(ys))/c(ys) + (L-d(ys))/d(ys) ) ) )/m,
    lambda t,ys: ys[3],
    lambda t,ys: ( k*  ( (L+ys[2]) *( L - c(ys)) / c(ys) - (L-ys[2])*(L - d(ys)) / d(ys) + \
                   ys[2] *( (L-a(ys))/a(ys) + (L-b(ys))/b(ys) ) ) )/m - g
]

In [ ]:
k = 25

In [ ]:
x_0 = 3
v_x_0 = 0
y_0 = 3
v_y_0 = 0

In [ ]:
spring_node(method=fehlberg_4_5,
              y_0s=[x_0,v_x_0,y_0,v_y_0],
              approx= functions,
              step=0.05)

In [ ]:
a = lambda ys: np.sqrt((L + ys[0])**2 + ys[2]**2)
b = lambda ys: np.sqrt((L - ys[0])**2 + ys[2]**2)
c = lambda ys: np.sqrt((ys[0]) + (L + ys[2])**2)
d = lambda ys: np.sqrt((ys[0]) + (L - ys[2])**2)
functions = [
    lambda t,ys: ys[1],
    lambda t,ys: ( k * ( (L+ys[0]) *( L - a(ys)) / a(ys) - (L-ys[0])*(L - b(ys)) / b(ys) + \
                   ys[0] *( (L-c(ys))/c(ys) + (L-d(ys))/d(ys) ) ) )/m - \
                   np.sign(ys[1]) * c_ * ys[1]**2,
    lambda t,ys: ys[3],
    lambda t,ys: ( k*  ( (L+ys[2]) *( L - c(ys)) / c(ys) - (L-ys[2])*(L - d(ys)) / d(ys) + \
                   ys[2] *( (L-a(ys))/a(ys) + (L-b(ys))/b(ys) ) ) )/m - g - \
                   np.sign(ys[3]) * c_ * ys[3]**2
]

In [ ]:
spring_node(method=fehlberg_4_5,
              y_0s=[x_0,v_x_0,y_0,v_y_0],
              approx= functions,
              step=0.05)

In [ ]:
balls = np.mgrid[0:5,0:5]
pos = balls*10.
vel = np.zeros(shape=(2,3,3))
print vel
pos[0,2,2] = 15.
print pos
step = 0.01


mass = pos[:,1:4,1:4]
north = pos[:,0:3,1:4]
south = pos[:,2:5,1:4]
west = pos[:,1:4,0:3]
east = pos[:,1:4,2:5]

plt.figure()
plt.scatter(mass[1].ravel(),mass[0].ravel())
plt.show()

spring_len = lambda direction: np.sqrt(np.sum((direction-mass)**2,axis = 0))
spring_n = spring_len(north)
spring_s = spring_len(south)
spring_e = spring_len(east)
spring_w = spring_len(west)
update_north = (spring_n-L)*(mass-north)/spring_n
update_south = (spring_s-L)*(mass-south)/spring_s
update_east = (spring_e-L)*(mass-east)/spring_e
update_west = (spring_e-L)*(mass-east)/spring_w
update_vel = step*k*(update_north + update_south + update_west + update_east)/m
vel += update_vel
update_pos = step*vel
pos[:,1:4,1:4] += update_pos

plt.figure()
plt.scatter(mass[1].ravel(),mass[0].ravel())
plt.show()
print mass

In [164]:
def spring_field(step = 0.01, filename = None):
    import matplotlib.animation as animation
    global state
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111)
    ax.grid()
    plt.ylim(-5,70)
    plt.xlim(-5,70)
    
    points, = ax.plot([],[], linestyle='None', marker='o', markersize = 10.)
    
    time_template = 'time = %.1fs'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    
    pos = np.mgrid[0:5,0:5]*10.
    vel = np.zeros_like(pos)
    pos[0,2,2] = 30.
    
    state = np.array([pos,vel])
    
    def init():
        points.set_data([],[])
    
        return points, time_text
    
    def animate(t):
        global state
        mass = state[0,:,1:4,1:4]
        north = state[0,:,0:3,1:4]
        south = state[0,:,2:5,1:4]
        west = state[0,:,1:4,0:3]
        east = state[0,:,1:4,2:5]

        spring_len = lambda direction: np.sqrt(np.sum((direction-mass)**2,axis = 0))
        spring_n = spring_len(north)
        spring_s = spring_len(south)
        spring_e = spring_len(east)
        spring_w = spring_len(west)
        update_north = (spring_n-L)*(mass-north)/spring_n
        update_south = (spring_s-L)*(mass-south)/spring_s
        update_east = (spring_e-L)*(mass-east)/spring_e
        update_west = (spring_w-L)*(mass-west)/spring_w
        update_vel = -step*k*(update_north + update_south + update_east + update_west)/m
        update_pos = step*state[1]
        state[1,:,1:4,1:4] += update_vel
        state[0,:,:,:] += update_pos
        
        points.set_data(state[0,1].ravel(),state[0,0].ravel())
        time_text.set_text(time_template % (t*step))
        return points, time_text
    
    ani = animation.FuncAnimation(fig, animate, np.arange(400),
                              interval=20, blit=True, init_func=init)

    if filename:
        ani.save(filename + '.mp4', fps=30)
    plt.show()

In [165]:
spring_field()

d:\anac\lib\site-packages\IPython\kernel\__main__.py:39: RuntimeWarning: invalid value encountered in true_divide
d:\anac\lib\site-packages\IPython\kernel\__main__.py:40: RuntimeWarning: invalid value encountered in true_divide


In [93]:
balls = np.mgrid[0:5,0:5]
pos = balls*10.
vel = np.zeros(shape=(2,3,3))
pos[0,2,2] = 15.

In [163]:
k = k_ = 2
L = 10
m = 2

In [1]:
spring_len = lambda direction,ballz: np.sqrt(np.sum((direction-ballz)**2,axis = 0))
def update_fun(tn,x):
    mass = x[0,:,1:4,1:4]
    north = x[0,:,0:3,1:4]
    south = x[0,:,2:5,1:4]
    west = x[0,:,1:4,0:3]
    east = x[0,:,1:4,2:5]
    
    spring_n = spring_len(north,mass)
    spring_s = spring_len(south,mass)
    spring_e = spring_len(east,mass)
    spring_w = spring_len(west,mass)
    
    update_north = (spring_n-L)*(mass-north)/spring_n
    update_south = (spring_s-L)*(mass-south)/spring_s
    update_east = (spring_e-L)*(mass-east)/spring_e
    update_west = (spring_w-L)*(mass-west)/spring_w
    
    result = np.zeros_like(x)
    result[1,:,1:4,1:4] = -k_*(update_north + update_south + \
                              update_east + update_west)/m

    result[0,:,:,:] = x[1,:,:,:]
    return result

In [2]:
x = np.array([pos,np.zeros_like(pos)])
print x
update_fun(0,x)

NameError: name 'np' is not defined

In [147]:
def calc_k(k,b):
    return np.sum(b[:,np.newaxis,np.newaxis,np.newaxis,np.newaxis]*k,axis=0)
calc_k(np.array([pos,pos]),np.arange(2)+3)

array([[[[   0.        ,    0.        ,    0.        ,    0.        ,    0.        ],
         [  70.        ,   67.13575684,  118.99817313,   43.63011297,   70.        ],
         [ 140.        ,  123.5946551 ,  173.51045294,  134.34014021,  140.        ],
         [ 210.        ,  185.7463928 ,  206.95146457,  202.30876241,  210.        ],
         [ 280.        ,  280.        ,  280.        ,  280.        ,  280.        ]],

        [[   0.        ,   70.        ,  140.        ,  210.        ,  280.        ],
         [   0.        ,   94.29845092,  160.0120237 ,  170.45865154,  280.        ],
         [   0.        ,  108.38530938,  171.97627494,  181.72571217,  280.        ],
         [   0.        ,  131.36346232,  164.65827568,  214.36501408,  280.        ],
         [   0.        ,   70.        ,  140.        ,  210.        ,  280.        ]]],


       [[[   0.        ,    0.        ,    0.        ,    0.        ,    0.        ],
         [  70.        ,   67.13575684,  118.998

In [148]:
calc_k(np.array([pos,np.zeros_like(pos)]),np.arange(2))

array([[[[  0.        ,   0.        ,   0.        ,   0.        ,   0.        ],
         [ 10.        ,   9.59082241,  16.99973902,   6.23287328,  10.        ],
         [ 20.        ,  17.6563793 ,  24.78720756,  19.1914486 ,  20.        ],
         [ 30.        ,  26.53519897,  29.56449494,  28.90125177,  30.        ],
         [ 40.        ,  40.        ,  40.        ,  40.        ,  40.        ]],

        [[  0.        ,  10.        ,  20.        ,  30.        ,  40.        ],
         [  0.        ,  13.47120727,  22.85886053,  24.35123593,  40.        ],
         [  0.        ,  15.48361563,  24.56803928,  25.96081602,  40.        ],
         [  0.        ,  18.7662089 ,  23.52261081,  30.62357344,  40.        ],
         [  0.        ,  10.        ,  20.        ,  30.        ,  40.        ]]],


       [[[  0.        ,   0.        ,   0.        ,   0.        ,   0.        ],
         [  0.        ,   0.        ,   0.        ,   0.        ,   0.        ],
         [  0.        

In [149]:
class SuperSystemSolver():
    def __init__(self,updates = None):
        if isinstance(updates,list):
            max_len = max([len(x) for x in updates])
            [x.extend([0]*(max_len-len(x))) for x in updates]
            self.updates = np.asarray(updates, dtype = np.float64)
        else:
            self.updates = updates
        
    @property
    def update_count(self):
        return min(self.updates.shape)-1
    
    def __call__(self,tn,x,function,h):
        k_x = np.zeros(shape=(self.update_count,)+x.shape)
        for i in range(self.update_count):
            a = self.updates[i,0]
            b = self.updates[i,1:]

            upd_x = h*calc_k(k_x,b)
            
            k_x[i] = function(tn+a*h,x+upd_x)
            
        c = self.updates[self.update_count,1:]
        return h*np.sum(calc_k(k_x,c), axis = 0)

In [150]:
merson = SuperSystemSolver(
    [[0],
     [1/3,1/3],
     [1/3,1/6,1/6],
     [1/2,1/8,0,3/8],
     [1,1/2,0,-3/2,2],
     [0,1/6,0,0,2/3,1/6]])

In [151]:
state = np.array([pos,np.zeros_like(pos)])

In [152]:
state_update = merson(0,np.array([pos,np.zeros_like(pos)]),update_fun,0.01) 
state = state + state_update
print state

[[[[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
      0.00000000e+00]
   [  1.00000000e+01   9.58574229e+00   1.68872515e+01   6.29819238e+00
      1.00000000e+01]
   [  2.00000000e+01   1.76859221e+01   2.47886161e+01   1.91290011e+01
      2.00000000e+01]
   [  3.00000000e+01   2.65843601e+01   2.96505354e+01   2.89021284e+01
      3.00000000e+01]
   [  4.00000000e+01   4.00000000e+01   4.00000000e+01   4.00000000e+01
      4.00000000e+01]]

  [[  0.00000000e+00   1.00000000e+01   2.00000000e+01   3.00000000e+01
      4.00000000e+01]
   [  0.00000000e+00   1.34469429e+01   2.28281747e+01   2.44026658e+01
      4.00000000e+01]
   [  0.00000000e+00   1.54418628e+01   2.45608626e+01   2.60114195e+01
      4.00000000e+01]
   [  0.00000000e+00   1.86080715e+01   2.35166398e+01   3.06427031e+01
      4.00000000e+01]
   [  0.00000000e+00   1.00000000e+01   2.00000000e+01   3.00000000e+01
      4.00000000e+01]]]


 [[[  0.00000000e+00   0.00000000e+00   0.00000000e+00

In [153]:
euler = SuperSystemSolver([
        [0],
        [0,1]
    ])

In [158]:
def spring_field_rk(step = 0.01, filename = None):
    import matplotlib.animation as animation
    global state
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111)
    ax.grid()
    plt.ylim(-5,70)
    plt.xlim(-5,70)
    
    points, = ax.plot([],[], linestyle='None', marker='o', markersize = 10.)
    
    time_template = 'time = %.1fs'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    
    pos = np.mgrid[0:5,0:5]*10.
    vel = np.zeros_like(pos)
    pos[0,2,2] = 23.
    
    state = np.array([pos,vel])
    
    def init():
        points.set_data([],[])
    
        return points, time_text
    
    def animate(t):
        global state
        
        state_update = merson(0,state,update_fun,0.01)
        state += state_update
        
        points.set_data(state[0,1,:,:].ravel(),state[0,0,:,:].ravel())
        time_text.set_text(time_template % (t*step))
        return points, time_text
    
    ani = animation.FuncAnimation(fig, animate, np.arange(400),
                              interval=20, blit=True, init_func=init)

    if filename:
        ani.save(filename + '.mp4', fps=30)
    plt.show()

In [159]:
spring_field_rk()